# Donkey Car Attempt

My attempts to build a CNN from scratch are hitting walls. I am going to try out a model suggested on the [Donkey Car Webpage](https://docs.donkeycar.com/dev_guide/model/), to establish a "baseline" (very broadly) of sorts to which I can compate the "custom" models of my own design.

Steps:
1. Establish modeling parameters
1. Split data into training and testing sets
1. Scale telemetry data with scikitlearn's minmaxscaler or standardscaler
1. Save the scaler to scale fresh driving input data for inferences
1. Define methods to construct, fit, plot, and save a model
1. Fit, save, and plot a constructed model with a range of batch sizes
1. Save the model and record path and metrics/parameters in a CSV file

In [1]:
## Imports
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import tensorflow as tf
# import keras_tuner as kt
import time
import pickle
import sklearn.metrics as metrics

from modeling_methods import run_model, plot_metrics, save_model, create_donkey_vimu

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from keras_tuner import BayesianOptimization, Hyperband, HyperModel

from os.path import exists

# from tensorflow import keras
# from tensorflow.keras.backend import concatenate
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, Flatten, Convolution2D
from tensorflow.keras.metrics import MAE, MSE, RootMeanSquaredError
# from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

2021-11-18 18:14:39.027021: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
## Directories
working_date = '11_12_2021'
working_time = '19_28_18'

## Directories
data_directory = f'../data/{working_date}/{working_time}'
model_directory = f'../models/{working_date}/{working_time}'

## File paths
cam_input_dataset_file = f'{data_directory}/X_img.npy'
telem_input_dataset_file = f'{data_directory}/X_telem.pkl'
target_dataset_file = f'{data_directory}/y.npy'

## Parameters
scaler = 'standard' # ['minmax', 'standard']
dual_outputs = True
batch_sizes = [32, 64, 128, 256, 512, 1024]
early_stop_patience = 5 # None for no stop
epochs = 250

create_model = create_donkey_vimu

## Data

### Load Datasets

In [3]:
## Load the datasets
X_cam = np.load(cam_input_dataset_file)
X_telem = pd.read_pickle(telem_input_dataset_file).to_numpy()
y = np.load(target_dataset_file)

## Split for dual output
if dual_outputs:
    y_steering = y[:, 0]
    y_throttle = y[:, 1]

## Check Shape
X_cam.shape

(115052, 64, 64, 1)

### Train-Test Split

In [4]:
if dual_outputs:
    datasets = train_test_split(X_cam, X_telem, y_steering, y_throttle, test_size=0.1, random_state=0)
else:
    datasets = train_test_split(X_cam, X_telem, y, test_size=0.1, random_state=0)
    
X_cam_train = datasets[0]
X_cam_test = datasets[1]
X_telem_train = datasets[2]
X_telem_test = datasets[3]
y_train = datasets[4]
y_test = datasets[5]
    
if dual_outputs:
    y_st_train = datasets[4]
    y_st_test = datasets[5]
    y_th_train = datasets[6]
    y_th_test = datasets[7]

### Scale IMU Data

In [5]:
sc = None
scaler_file = None
if scaler == 'minmax':
    sc = MinMaxScaler() # default range: [0, 1]
    scaler_file = f'{data_directory}/mm_scaler_{time.strftime("%m_%d_%H_%M")}.pkl'
elif scaler == 'standard':
    sc = StandardScaler()
    scaler_file = f'{data_directory}/ss_scaler_{time.strftime("%m_%d_%H_%M")}.pkl'

## Fit to then and transform training data
X_telem_train_sc = sc.fit_transform(X_telem_train)
## Transform testing data
X_telem_test_sc = sc.transform(X_telem_test)

### Save the Scaler for Predictions

In [6]:
## Save as pickle
pickle.dump(sc, open(scaler_file, 'wb'))

## Print path
scaler_file

'../data/11_12_2021/19_28_18/ss_scaler_11_18_18_14.pkl'

### Get Input Shape(s)

In [7]:
## Create variables
cam_input_shape = X_cam_train[0].shape
telem_input_shape = X_telem_train_sc[0].shape

## Check telemetry input shape
telem_input_shape

(7,)

## Model Loop

In [12]:
X_train = [X_cam_train, X_telem_train_sc]
X_test = (X_cam_test, X_telem_test_sc)

if dual_outputs:
    y_train = (y_st_train, y_th_train)
    y_test = (y_st_test, y_th_test)
else:
    y_train = y_train
    y_test = y_test

## Run models for each batch size
print('---')
for batch_size in batch_sizes:
    print(f'Batch size {batch_size} start: {time.strftime("%H:%M:%S")}')
    model = create_model(cam_input_shape, telem_input_shape, dual_outputs)
    model.compile(loss='mse', 
                  optimizer=Adam(learning_rate=0.0001), 
                  metrics=['mae', RootMeanSquaredError()])
    model, results = run_model(model, X_train, y_train, X_test, y_test, 
                               batch_size, epochs,
                              early_stop_patience=early_stop_patience)
    model_path = save_model(model_directory, model, results, batch_size, scaler_file)
    history = {k: v for k, v in results.history.items()}
    plot_metrics(history, batch_size)
    print(f'Batch size {batch_size} end:   {time.strftime("%H:%M:%S")}')
    print(f'Epochs run: {len(history["loss"])}')
    print(f'path: {model_path}')
    print('---')
# model_history.tail(len(batch_sizes))

---
Batch size 32 start: 18:14:41


2021-11-18 18:14:41.521345: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-18 18:14:41.522035: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-18 18:14:41.547578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 18:14:41.547703: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.797GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-11-18 18:14:41.547740: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-11-18 18:14:41.549030: I tensorflow/stream_executor/platfor

NameError: name 'model_directory' is not defined